In [1]:
import pandas as pd
import json

In [5]:
datos_json = []

with open('./json/output_steam_games.json', 'r') as archivo:
    for linea in archivo:
        try:
            objeto_json = json.loads(linea)
            datos_json.append(objeto_json)
        except json.JSONDecodeError as e:
            print(f"Error al cargar la línea: {e}")

In [6]:
df = pd.DataFrame(datos_json)
df.sample(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
113159,NaN,"[Casual, Simulation]",Rocksmith® 2014 – The Black Keys - “Your Touch”,Rocksmith® 2014 – The Black Keys - “Your Touch”,http://store.steampowered.com/app/390393/Rocks...,2015-11-24,"[Casual, Simulation]",http://steamcommunity.com/app/390393/reviews/?...,"[Single-player, Shared/Split Screen, Downloada...",2.99,False,390393,Ubisoft - San Francisco
78323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


procedemos a correr una funcion que nos permita encontrar valores nulos y campos vacios en nuestro dataframe

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


corremos nuestra funcion para revisar campos nulos y vacios

In [8]:
def analizar_datos(dataframe):
    num_filas, num_columnas = dataframe.shape
   
    resultados = {}
    
    for columna in dataframe.columns:
        nulos = dataframe[columna].isnull().sum()
        vacios = dataframe[columna].apply(lambda x: x == '').sum()
        
        resultados[columna] = {
            'filas_nulas': nulos,
            'campos_vacios': vacios
        }
    
    return resultados

In [9]:
resultados_analisis = analizar_datos(df) 
resultados_analisis

{'publisher': {'filas_nulas': 96362, 'campos_vacios': 0},
 'genres': {'filas_nulas': 91593, 'campos_vacios': 0},
 'app_name': {'filas_nulas': 88312, 'campos_vacios': 0},
 'title': {'filas_nulas': 90360, 'campos_vacios': 0},
 'url': {'filas_nulas': 88310, 'campos_vacios': 0},
 'release_date': {'filas_nulas': 90377, 'campos_vacios': 0},
 'tags': {'filas_nulas': 88473, 'campos_vacios': 0},
 'reviews_url': {'filas_nulas': 88312, 'campos_vacios': 0},
 'specs': {'filas_nulas': 88980, 'campos_vacios': 0},
 'price': {'filas_nulas': 89687, 'campos_vacios': 0},
 'early_access': {'filas_nulas': 88310, 'campos_vacios': 0},
 'id': {'filas_nulas': 88312, 'campos_vacios': 0},
 'developer': {'filas_nulas': 91609, 'campos_vacios': 0}}

procederemos a evaluar columnas con campos iguales

In [10]:
def verificar_duplicados(df, columna1, columna2):
   return df.duplicated([columna1, columna2]).any()

In [11]:
resultado = verificar_duplicados(df, 'app_name', 'title')
resultado

True

In [12]:
resultado = verificar_duplicados(df, 'url', 'reviews_url')
resultado

True

nos genera error las siguientes ya que estan los datos en tipos listas

In [ ]:
# resultado = verificar_duplicados(df, 'genres', 'tags')
# resultado

procedemos a eliminar campos como Nan ya que hay demasiada cantidad de campos, para asi obtener un dataframe mucho mas claro y conciso

In [14]:
df = df.dropna()

procederemos a crear una funcion que nos permita eliminar caracteres no deseados en nuestro data y hacer algo de etl

In [15]:
def limpiar_columnas(dataframe, columnas):
    for columna in columnas:
        # Convertir elementos de tipo lista a cadenas de texto
        dataframe[columna] = dataframe[columna].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
        
        # Eliminar corchetes y convertir texto a minúsculas con la primera letra en mayúscula
        dataframe[columna] = dataframe[columna].str.strip('[]').str.lower().str.capitalize()
        
    return dataframe

In [16]:
columnas = ['genres', 'tags','specs'] 
df = limpiar_columnas(df, columnas)

/tmp/ipykernel_5689/2676801907.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[columna] = dataframe[columna].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
/tmp/ipykernel_5689/2676801907.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[columna] = dataframe[columna].str.strip('[]').str.lower().str.capitalize()


se puede lograr analizar claramente que las columna app_name y title poseen los mismos valores

procedemos a eliminar app-name

In [17]:
df = df.drop('app_name', axis=1)

empezaremos a convertir tipos de datos como id, release_date y price para que sean datos de mas facil manejo a la hora de trabajar las querys de nuestra API

In [18]:
def limpiar_caracteres(dataframe, columnas):
    caracteres_a_limpiar = ["'", '"', ',', '.']
    
    for columna in columnas:
        for caracter in caracteres_a_limpiar:
            dataframe[columna] = dataframe[columna].replace(caracter, '', regex=True)
    
    return dataframe

In [19]:
columnas = ['price'] 
df = limpiar_caracteres(df, columnas)

In [20]:
df['price'].unique()

array([4.99, '', 0.99, 3.99, 18.99, 29.99, 10.99, 2.99, 1.59, 14.99, 1.99,
       59.99, 9.99, 6.99, 7.99, 39.99, 19.99, 7.49, 8.99, 12.99, 5.99,
       2.49, 15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 34.99,
       1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99, 49.99, 13.98,
       149.99, 771.71, 21.99, 89.99, 0.98, 139.92, 4.29, 54.99, 64.99,
       74.99, 0.89, 0.5, 299.99, 1.29, 119.99, 44.99, 3.0, 15.0, 1.39,
       2.0, 4.0, 1.95, 1.5, 6.66, 26.99, 399.99, 31.99, 20.0, 40.0, 5.0,
       3.33, 38.85, 71.7, 995.0, 5.49, 27.49, 3.39, 6.0, 19.95, 20.99,
       499.99, 27.99, 199.99, 4.68, 131.4, 44.98, 202.76, 2.3, 0.95,
       36.99, 172.24, 249.99, 2.97, 10.96, 2.66, 6.48, 10.0, 1.0, 11.15,
       49.0, 199.0, 99.0, 87.94, 0.49, 9.98, 9.95, 12.89, 6.49, 1.87,
       41.99, 4.49, 289.99, 23.96, 5.65, 12.0, 129.99, 23.99, 13.37,
       189.96, 59.95, 160.91], dtype=object)

In [21]:
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['price'].fillna(0, inplace=True)

In [33]:
df.to_parquet('output_steam_games.parquet')
df = pd.read_parquet('./parquet/output_steam_games.parquet')

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22530 non-null  object 
 1   genres        22530 non-null  object 
 2   title         22530 non-null  object 
 3   url           22530 non-null  object 
 4   release_date  22530 non-null  object 
 5   tags          22530 non-null  object 
 6   reviews_url   22530 non-null  object 
 7   specs         22530 non-null  object 
 8   price         22530 non-null  float64
 9   early_access  22530 non-null  bool   
 10  id            22530 non-null  object 
 11  developer     22530 non-null  object 
dtypes: bool(1), float64(1), object(10)
memory usage: 2.1+ MB


corremos una funcion para repasar espacios en blanco entre las cadenas

In [34]:
def limpiar_columnas(df):
    for columna in df.columns:
        if df[columna].dtype == 'object': 
            df[columna] = df[columna].str.strip() 
            df[columna] = df[columna].str.replace('"', '').str.replace("'", '')  
            df[columna] = df[columna].str.lower().str.capitalize()  
    
    return df

In [35]:
df = limpiar_columnas(df)

In [62]:
import numpy as np

def comprobar(df):
    duplicados = df[df.duplicated(keep=False)]
    vacios = df.apply(lambda x: (x.astype(str).str.strip() == '').any() if x.dtype == 'object' else False, axis=0)
    
    valores_nulos = df.isnull().sum()
    valores_nan = df.isna().sum()

    return {
        'duplicados': duplicados,
        'campos_vacios': vacios,
        'valores_nulos': valores_nulos,
        'valores_nan': valores_nan
    }

In [63]:
resultado = comprobar(df)
resultado

{'duplicados': Empty DataFrame
 Columns: [publisher, genres, title, url, release_date, tags, reviews_url, specs, price, early_access, id, developer]
 Index: [],
 'campos_vacios': publisher       False
 genres          False
 title           False
 url             False
 release_date    False
 tags            False
 reviews_url     False
 specs           False
 price           False
 early_access    False
 id              False
 developer       False
 dtype: bool,
 'valores_nulos': publisher       0
 genres          0
 title           0
 url             0
 release_date    0
 tags            0
 reviews_url     0
 specs           0
 price           0
 early_access    0
 id              0
 developer       0
 dtype: int64,
 'valores_nan': publisher       0
 genres          0
 title           0
 url             0
 release_date    0
 tags            0
 reviews_url     0
 specs           0
 price           0
 early_access    0
 id              0
 developer       0
 dtype: int64}

In [72]:
df.sample(5)

,publisher,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
109608,Thought ops llc,"Action, adventure, casual, free to play, indie...",Rescue bear operation,Http://store.steampowered.com/app/513210/rescu...,2016-09-29,"Early access, free to play, indie, action, adv...",Http://steamcommunity.com/app/513210/reviews/?...,"Single-player, steam achievements, full contro...",0.00,True,513210,Cascade game foundry
101584,"Mcleodgaming inc., world entertainment studios...","Action, indie, strategy","Yeah jam fury: u, me, everybody!",Http://store.steampowered.com/app/633930/yeah_...,2017-12-15,"Strategy, action, indie, puzzle-platformer, pl...",Http://steamcommunity.com/app/633930/reviews/?...,"Single-player, local co-op, steam achievements...",11.99,False,633930,"World entertainment studios, llc,mcleodgaming ..."
114610,Snk corporation,Action,Metal slug,Http://store.steampowered.com/app/366250/metal...,2015-06-25,"Action, arcade, 2d, side scroller, shoot em up...",Http://steamcommunity.com/app/366250/reviews/?...,"Single-player, multi-player, co-op, steam achi...",7.99,False,366250,Dotemu
89689,Born ready games,"Action, indie, simulation",Strike suit zero heroes of the fleet dlc,Http://store.steampowered.com/app/209548/strik...,2013-05-20,"Action, indie, simulation",Http://steamcommunity.com/app/209548/reviews/?...,"Single-player, downloadable content, steam ach...",6.99,False,209548,Born ready games ltd.
106941,Nexon america,"Adventure, free to play, massively multiplayer...",Riders of icarus - heroic veiled corruption pa...,Http://store.steampowered.com/app/606320/rider...,2017-03-22,"Adventure, free to play, rpg, massively multip...",Http://steamcommunity.com/app/606320/reviews/?...,"Single-player, multi-player, online multi-play...",29.99,False,606320,Wemade


In [73]:
# df.to_parquet('output_steam_games.parquet')